## **Use Case Overview**

Imagine you’re part of an R&D team that needs to merge structured data (e.g., experimental results, market trends) from Microsoft Fabric with unstructured documents (e.g., research reports, engineering notes) in SharePoint, then validate these findings against external references (e.g., Bing) and a high-quality “ground truth” internal knowledge stores (e.g., Azure AI Search).

This was a classic Retrieval-Augmented Generation (RAG) scenario—multiple data sources must be queried in real time and cross-checked for consistency. However, by leveraging Azure AI agent services (an agetic enterprise-ready microservices approach) alongside frameworks like Semantic Kernel, we can evolve beyond basic RAG into a mostly autonomous, agentic system. In this design, Agentic RAG and the Reflection Pattern enable each agent to iteratively refine its output until it’s confident in delivering a high-quality, validated answer—paving the way for intelligent automation that continually learns and improves.

To summarize, you’re not only bringing data to the AI but also bringing AI to the data, thus maximizing the value of your knowledge stores. By leveraging state-of-the-art retrieval solutions like Azure AI Search, while also tapping sources such as SharePoint (unstructured data) and Fabric (structured data), you can harness your most valuable asset—data—to achieve new levels of insight and automation. 

**In this demo, we have two Azure AI Agents (extending beyond a single-agent architecture):**

+ DataRetrievalAgent: Has access to Microsoft Fabric (for structured data) and SharePoint (for unstructured documents). Its job is to gather relevant internal data: for example, “failure rates of Material X in high-temperature tests,” or “engineering notes on prior tests.”

- ValidationInsightsAgent Has access to Bing / Azure Cognitive Search for external references and can run a “reflection” or “validation” step. Its job is to cross-check what was returned by the first agent and highlight missing or conflicting information. ValidationInsightsAgent has access to highly curated knowledge sources (e.g., Azure AI Search) for validating the accuracy or truthfulness of the information it receives from DataRetrievalAgent.


**Moving from a single-agent setup to a multi-agent system is now simpler than ever with Semantic Kernel. The general flow looks like this:**

1. The user asks a question (e.g., “Retrieve historical failure rates for Material X in extreme temperatures and cross-check if new standards or conflicting data exist.”).
2. The DataRetrievalAgent fetches structured data from Fabric (e.g., lab test results, analytics) and unstructured docs from SharePoint (e.g., research memos, engineering notes).
3. The ValidationInsightsAgent then queries Bing/Azure Search to verify or supplement the results. Employ a reflection pattern, where it iterates over the combined results, looking for gaps or inconsistencies. If needed, it loops back to the DataRetrievalAgent for clarifications or additional data.

Finally, the user receives a validated, summarized answer that merges internal data with external cross-checks. Thanks to the agents’ back-and-forth reflection.

### **Why This Matters**

+ **Reduced Manual Research**: Instead of manually sifting through multiple data silos and external search engines, the AI Agents automate data gathering and vetting.
+ **Higher Confidence**: Validation ensures data accuracy and highlights missing pieces, improving R&D decision-making.
+ **Enterprise-Grade Security**: Each agent can enforce On-Behalf-Of (OBO) authentication to protect sensitive data (e.g., only pulling data the user is authorized to see).
In the Jupyter Notebook

When you run the code in this Jupyter notebook:

(TODO)

In [2]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio

# Azure AI Projects

import asyncio

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

# helper functions
from utils.utilityfucntions import print_agent_summary

# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.


In [3]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential())

### **1. Creating Azure AI Agents: DataRetrievalAgent**

The DataRetrievalAgent is responsible for internal data retrieval, combining structured data from Microsoft Fabric with unstructured documents from SharePoint. This agent ensures that research teams can efficiently access critical R&D insights, such as historical failure rates, experimental results, and engineering notes—all while maintaining secure and authorized access controls.

Agent Capabilities
+ ✅ Structured Data Retrieval → Queries Microsoft Fabric for experiment logs, test results, and structured analytics.
+ ✅ Unstructured Document Search → Fetches relevant reports, blueprints, and research notes from SharePoint.
+ ✅ OBO Authentication → Uses On-Behalf-Of (OBO) authentication to ensure users can only access data they are permitted to view.

For a detailed breakdown of how to create a single Azure AI Agent and configure its data (tools) connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).

In [7]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

In [5]:
from azure.ai.projects.models import (
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # # Retrieve and add Fabric Tool
    # fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    # toolset.add(FabricTool(connection_id=fabric_connection.id))

    # logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 00:27:36,984 - micro - MainProcess - ERROR    Failed to create ToolSet: name 'get_connection_id' is not defined (4254263396.py:<module>:23)
ERROR:micro:Failed to create ToolSet: name 'get_connection_id' is not defined


NameError: name 'get_connection_id' is not defined

In [ ]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="DataRetrievalAgent",
    description=(
        "An AI agent designed to retrieve and integrate structured data from Microsoft Fabric "
        "and unstructured documents from SharePoint to provide comprehensive R&D insights."
    ),
    instructions=(
        "You are an AI assistant specialized in retrieving data from internal enterprise sources. "
        "Utilize the provided tools to access and integrate information from Microsoft Fabric and SharePoint. "
        "Ensure that all responses are based on the most relevant and recent data available."
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Internal Data Retrieval for R&D",
        "data_source": "Microsoft Fabric and SharePoint",
        "response_validation": "Ensure data accuracy and relevance"
    },
)

# Print the agent's run ID (agent ID)
print(f"DataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

DataRetrievalAgent Run ID: asst_Wo0GJ9MpmvkfRPNwllC7bYFS


In [9]:
from semantic_kernel.contents import AuthorRole
from azure.core.exceptions import HttpResponseError
from datetime import timedelta
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

dataretrievalagent = AzureAIAgent(
    client=project_client,
    definition=dataretrievalagent_settings_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

thread = await project_client.agents.create_thread()

USER_INPUTS = [
    "What are the latest trends in R&D?",
    "What are the key features from Dexcom G7 CGM System?",
    "What documents are available in SharePoint related to R&D?",
    "How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?"]
try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await dataretrievalagent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in dataretrievalagent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logging.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the latest trends in R&D?

🤖 **Agent:** The latest trends in research and development (R&D) in 2023 highlight significant advancements across various fields. Some of the notable trends include:

1. **Diabetes Technology**:
   - The approval of the Omnipod 5 automated insulin delivery (AID) system, which is a hybrid closed-loop insulin delivery system that uses a tubeless pod insulin pump connected to a continuous glucose monitoring (CGM) sensor via Bluetooth. This system helps modulate insulin delivery based on real-time glucose readings【9:1†source】.
   - The clearance of the MiniMed 780G advanced hybrid closed-loop system with Guardian 4 sensor, focusing on improving glycemic outcomes for individuals with diabetes【7:1†source】.
   - Development and approval of the Omnipod GO, a basal-only insulin pod designed to simplify insulin delivery for people with type 2 diabetes【7:1†source】.
   - The increase in the use of diabetes devices and the need for healthcare profess

### **2. Creating Azure AI Agents: ValidationInsightsAgent**

The ValidationInsightsAgent is designed to validate and cross-check the data retrieved by the DataRetrievalAgent. It accesses external references such as Bing and Azure Cognitive Search to verify and supplement the internal data. Additionally, it leverages highly curated knowledge sources (e.g., Azure AI Search) to ensure the accuracy and truthfulness of the information, using a reflection or validation step to highlight missing or conflicting details.

Agent Capabilities:

+ ✅ External Reference Verification → Queries Bing and Azure Cognitive Search for real-time validation.
+ ✅ Reflection & Validation Step → Iteratively reviews and refines the information received from the DataRetrievalAgent.
+ ✅ Curated Knowledge Validation → Uses Azure AI Search to confirm the accuracy and reliability of internal data.

For a detailed breakdown of how to create a ValidationInsightsAgent and configure its external tools and connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).


In [9]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

    pdf_file_path = r"C:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf"
    logger.info(f"Using PDF file path: {pdf_file_path}")

    file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )

    # 2. Create file search tool with uploaded resources
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    toolset.add(file_search)
    logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 00:29:31,800 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/agentsbinggrounding (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/agentsbinggrounding
2025-03-20 00:29:31,802 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (3079022251.py:<module>:19)
INFO:micro:Bing Grounding Tool added successfully.
2025-03-20 00:29:31,805 - micro - MainProcess - INFO     Using PDF file path: C:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf (3079022251.py:<module>:31)
INFO:micro:Using PDF file pa

In [10]:
validationinsightsagent_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="ValidationInsightsAgent",
    description=(
        "An AI agent designed to validate and refine R&D insights by cross-checking internal and external data sources. "
        "It employs a reflection pattern to ensure response accuracy and relevance."
    ),
    instructions=(
        "You are a validation AI assistant specializing in cross-checking and enhancing insights from internal enterprise data and external sources. "
        "Prioritize using the internal file search index (vector store) for information retrieval whenever available. "
        "Leverage tools like Bing, Azure AI Search, and SharePoint to validate and supplement the data. "
        "Incorporate a reflection step to evaluate data quality, resolve inconsistencies, and refine your responses with proper citations. "
        "Deliver responses that are accurate, actionable, and insightful."
    ),
    toolset=toolset,
    tool_resources=file_search.resources,  # Always use file search vector store tool resources
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Cross-Validation and Insight Generation for R&D",
        "data_source": "Internal (Microsoft Fabric, SharePoint) and External (Bing, Azure AI Search)",
        "response_validation": "Include a reflection step to validate data accuracy and provide citations"
    },
)

# Print the new agent's run ID
print(f"ValidationInsightsAgent Run ID: {validationinsightsagent_definition.id}")

ValidationInsightsAgent Run ID: asst_6UraIHhjFuopzUXLcwovSleF


In [17]:
validation_agent = AzureAIAgent(
    client=project_client,
    definition=validationinsightsagent_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

# Create a conversation thread for the agent
thread = await project_client.agents.create_thread()

# Define a list of user inputs designed to trigger both internal search (e.g., product architecture) 
# and external market trend validation.
USER_INPUTS = [
    "What are the characteristics and architecture of Product A?",
    "What are the key features from Dexcom G7 CGM System?",
]

try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await validation_agent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in validation_agent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the characteristics and architecture of Product A?

🤖 **Agent:** The document provided is protected by Microsoft Information Protection and cannot be accessed directly. To proceed, please provide access permissions or use a PDF viewer that supports Azure Rights Management.

👤 **User:** What are the key features from Dexcom G7 CGM System?

🤖 **Agent:** It appears that the document related to Dexcom G7 CGM System is also protected by Microsoft Information Protection and cannot be accessed directly. To proceed, please provide access permissions or use a PDF viewer that supports Azure Rights Management.

